In [12]:
# ライブラリ
import requests
import datetime
import csv
import os
import shutil
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from time import sleep

# 初期化
url = 'https://python-data-collection.herokuapp.com/mens-fashion'
parser = 'html.parser'
item_list = []

# 練習サイトの一覧ページ(10ページ分)をループ処理
for i in range(9):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, parser)
    
    # 一覧ページ上の各商品のデータ取得
    for elem in soup.find_all('div', class_='col-md-2'):
        item_image_url = urljoin(url, elem.find('img')['src'])
        item_name = elem.find('div', class_='card-body').text.split('\n')[2]
        item_brand = elem.find('div', class_='card-body').text.split('\n')[3]
        item_price = int(elem.find('div', class_='card-body').text.split('\n')[4].replace(',', '')[1:])
        item_list.append([item_name, item_image_url, item_brand, item_price])
    
    # 次ページのURL更新
    if soup.select("a:contains('Next')"):
        url = urljoin(url, soup.select("a:contains('Next')")[0]['href'])
        sleep(1)
    else:
        break

# CSVファイル
csv_head = ['商品名', '画像URL', 'ブランド', '価格']
csv_date = datetime.datetime.today().strftime('%Y%m%d%H')
csv_file = 'mens_fashion_' + csv_date + '.csv'

# CSVファイルにデータ保存
with open(csv_file, 'w', errors='ignore') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerow(csv_head)
    writer.writerows(item_list)

# 画像ファイルの保存先
path = r'./download'

# 画像ファイルを保存
for item in item_list:
    file_name = item[0] + '.jpg'
    image_path = os.path.join(path, file_name)
    response = requests.get(item[1], stream=True)
    with open(image_path, 'wb') as file:
        shutil.copyfileobj(response.raw, file)